In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

# import os
# for dirname, _, filenames in os.walk('/kaggle/input'):
#     for filename in filenames:
#         print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

file_path = '/kaggle/input/digico-ai/train.csv'
df = pd.read_csv(file_path)

column_ratio = df.apply(lambda x: x.value_counts().max() / len(df))
columns_to_drop_equal = column_ratio[column_ratio >= 0.95].index
# print(columns_to_drop)
df_cleaned_equal = df.drop(columns=columns_to_drop_equal, inplace=False)

nan_counts = df_cleaned_equal.isnull().sum()
columns_to_drop_nan = nan_counts[nan_counts >= 0.4 * len(df)].index
# print(columns_to_drop_nan)
df_cleaned_nan = df_cleaned_equal.drop(columns=columns_to_drop_nan, inplace=False)
# df_cleaned_nan.head()
# print(df_cleaned_nan.isnull().sum())
# print(df_cleaned_nan.mean())
df_filled = df_cleaned_nan.fillna(df_cleaned_nan.mean())
train_mean = df_filled.mean()


/tmp/ipykernel_20/3530840875.py:34: FutureWarning: The default value of numeric_only in DataFrame.mean is deprecated. In a future version, it will default to False. In addition, specifying 'numeric_only=None' is deprecated. Select only valid columns or specify the value of numeric_only to silence this warning.
  df_filled = df_cleaned_nan.fillna(df_cleaned_nan.mean())
/tmp/ipykernel_20/3530840875.py:35: FutureWarning: The default value of numeric_only in DataFrame.mean is deprecated. In a future version, it will default to False. In addition, specifying 'numeric_only=None' is deprecated. Select only valid columns or specify the value of numeric_only to silence this warning.
  train_mean = df_filled.mean()


In [2]:
df_filled['Label'] = df_filled['Label'].replace({'normal':0, 'defect':1})

from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler

input_column_names = df_filled.columns[1:].tolist()
scaler= StandardScaler()
df_filled[input_column_names] = scaler.fit_transform(df_filled[input_column_names])
# # 학습 데이터에 대해 fit_transform 수행
# scaler = StandardScaler()
# df_train[columns_to_normalize] = scaler.fit_transform(df_train[columns_to_normalize])

# # 예측 데이터에 대해 transform만 수행
# df_test[columns_to_normalize] = scaler.transform(df_test[columns_to_normalize])


/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [3]:
import tensorflow as tf
from tensorflow import keras
from sklearn.model_selection import train_test_split

X = df_filled.iloc[:, 1:].values
y = df_filled.iloc[:, 0].values

X_train, X_test, y_train, y_test = train_test_split(X, y, 
                                                    test_size=0.2, 
                                                    random_state=42)

model = keras.Sequential([
    keras.layers.Dense(256, activation='relu', input_shape=(X.shape[1],)),
    keras.layers.Dense(128, activation='relu'),
    keras.layers.Dense(64, activation='relu'),
    keras.layers.Dense(32, activation='relu'),
    keras.layers.Dense(32, activation='relu'),
    keras.layers.Dense(1, activation='sigmoid')
])

# 모델 컴파일
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

early_stopping_callback = keras.callbacks.EarlyStopping(monitor='val_loss', patience=5, mode='min', verbose=1)
# 모델 학습
model.fit(X_train, y_train, epochs=20, batch_size=32, validation_split=0.2, callbacks=[early_stopping_callback])

# 모델 평가
test_loss, test_acc = model.evaluate(X_test, y_test)
print("Test accuracy:", test_acc)

/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:98: UserWarning: unable to load libtensorflow_io_plugins.so: unable to open file: libtensorflow_io_plugins.so, from paths: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so']
caused by: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so: undefined symbol: _ZN3tsl6StatusC1EN10tensorflow5error4CodeESt17basic_string_viewIcSt11char_traitsIcEENS_14SourceLocationE']
  warnings.warn(f"unable to load libtensorflow_io_plugins.so: {e}")
/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:104: UserWarning: file system plugins are not loaded: unable to open file: libtensorflow_io.so, from paths: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io.so']
caused by: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io.so: undefined symbol: _ZTVN10tenso

Epoch 1/20
100/100 [==============================] - 2s 6ms/step - loss: 0.1361 - accuracy: 0.9513 - val_loss: 0.0442 - val_accuracy: 0.9837
Epoch 2/20
100/100 [==============================] - 0s 4ms/step - loss: 0.0236 - accuracy: 0.9919 - val_loss: 0.0225 - val_accuracy: 0.9912
Epoch 3/20
100/100 [==============================] - 0s 4ms/step - loss: 0.0175 - accuracy: 0.9959 - val_loss: 0.0326 - val_accuracy: 0.9912
Epoch 4/20
100/100 [==============================] - 0s 4ms/step - loss: 0.0025 - accuracy: 0.9994 - val_loss: 0.0347 - val_accuracy: 0.9950
Epoch 5/20
100/100 [==============================] - 0s 4ms/step - loss: 2.1019e-04 - accuracy: 1.0000 - val_loss: 0.0372 - val_accuracy: 0.9950
Epoch 6/20
100/100 [==============================] - 0s 4ms/step - loss: 1.0280e-04 - accuracy: 1.0000 - val_loss: 0.0389 - val_accuracy: 0.9950
Epoch 7/20
100/100 [==============================] - 0s 4ms/step - loss: 6.5320e-05 - accuracy: 1.0000 - val_loss: 0.0402 - val_accuracy: 0

In [4]:
file_path_ = '/kaggle/input/digico-ai/test.csv'
df_ = pd.read_csv(file_path_)
# print(df_.isnull().sum())

df_cleaned_equal_ = df_.drop(columns=columns_to_drop_equal, inplace=False)
# print(df_cleaned_equal_)
df_cleaned_nan_ = df_cleaned_equal_.drop(columns=columns_to_drop_nan, inplace=False)
# print(df_cleaned_nan_)
df_filled_ = df_cleaned_nan_.fillna(train_mean)
# print(df_filled_)

from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler

input_column_names = df_filled_.columns[0:].tolist()
# print(input_column_names)
df_filled_ = scaler.transform(df_filled_[input_column_names])
# # 학습 데이터에 대해 fit_transform 수행
# scaler = StandardScaler()
# df_train[columns_to_normalize] = scaler.fit_transform(df_train[columns_to_normalize])

# # 예측 데이터에 대해 transform만 수행
# df_test[columns_to_normalize] = scaler.transform(df_test[columns_to_normalize])

# X_pred = df_filled_.iloc[:,].values
# print(X_pred)

predictions = model.predict(df_filled_)
indexs = list(range(0, len(predictions)))
binary_predictions = (predictions > 0.5).astype(int)

# print(len(indexs), " ", len(binary_predictions.ravel()))
results = pd.DataFrame({'ID': indexs, 'Target': binary_predictions.ravel()})
results.to_csv('prediction_results.csv', index=False)

180/180 [==============================] - 0s 1ms/step
